In [2]:
import os
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments
from transformers import pipeline
from datasets import load_dataset
from sklearn.metrics import accuracy_score

In [3]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric
train_dataset = load_dataset("imdb",split="train[:20000]")
val_dataset = load_dataset("imdb",split="train[20000:25000]")
test_dataset=load_dataset("imdb",split="test[:5000]")
print(train_dataset.features)

Using the latest cached version of the module from C:\Users\LEOVO\.cache\huggingface\modules\datasets_modules\datasets\imdb\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a (last modified on Wed Aug 11 17:01:21 2021) since it couldn't be found locally at imdb\imdb.py or remotely (ConnectionError).
Using the latest cached version of the module from C:\Users\LEOVO\.cache\huggingface\modules\datasets_modules\datasets\imdb\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a (last modified on Wed Aug 11 17:01:21 2021) since it couldn't be found locally at imdb\imdb.py or remotely (ConnectionError).
Reusing dataset imdb (C:\Users\LEOVO\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)
Using the latest cached version of the module from C:\Users\LEOVO\.cache\huggingface\modules\datasets_modules\datasets\imdb\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a (last modified on Wed Aug 11

{'text': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['neg', 'pos'], names_file=None, id=None)}


In [4]:
train_dataset = train_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
train_dataset[0]

Loading cached processed dataset at C:\Users\LEOVO\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a\cache-d2923ef276bb2dbe.arrow


{'label': 1,
 'labels': 1,
 'text': 'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'}

In [5]:
val_dataset = val_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
test_dataset = test_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)

Loading cached processed dataset at C:\Users\LEOVO\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a\cache-b84e22827b727556.arrow
Loading cached processed dataset at C:\Users\LEOVO\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a\cache-acc648cae1853c46.arrow


In [6]:
model_id = 'prajjwal1/bert-tiny'
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=train_dataset.features["label"].num_classes)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

In [7]:
MAX_LENGTH = 256
train_dataset = train_dataset.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
val_dataset = val_dataset.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
test_dataset = test_dataset.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)


Loading cached processed dataset at C:\Users\LEOVO\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a\cache-5e00ee5c23f0c28d.arrow
Loading cached processed dataset at C:\Users\LEOVO\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a\cache-6e5cb76d06ab58fc.arrow
Loading cached processed dataset at C:\Users\LEOVO\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a\cache-3957f7269d52a9f8.arrow


In [8]:
train_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [9]:
train_dataset.features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': ClassLabel(num_classes=2, names=['neg', 'pos'], names_file=None, id=None),
 'labels': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [10]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [11]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    learning_rate=3e-4,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    do_train=True,
    do_eval=True,
    no_cuda=False,
    load_best_model_at_end=True,
    # eval_steps=100,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics
)

train_out = trainer.train()

 11%|█         | 101/939 [00:18<02:19,  6.02it/s]

{'loss': 0.5451, 'learning_rate': 0.0002680511182108626, 'epoch': 0.32}


 21%|██▏       | 201/939 [00:34<02:04,  5.93it/s]

{'loss': 0.382, 'learning_rate': 0.00023610223642172524, 'epoch': 0.64}


 32%|███▏      | 301/939 [00:51<01:46,  5.98it/s]

...

{'loss': 0.1577, 'learning_rate': 1.2460063897763577e-05, 'epoch': 2.88}


100%|█████████▉| 938/939 [02:54<00:00,  6.14it/s]D:\coding\python\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

100%|██████████| 939/939 [03:00<00:00,  6.14it/s]

{'eval_loss': 0.5265847444534302, 'eval_accuracy': 0.8222, 'eval_f1': 0.45121281966853255, 'eval_precision': 0.5, 'eval_recall': 0.4111, 'eval_runtime': 5.9288, 'eval_samples_per_second': 843.334, 'epoch': 3.0}


100%|██████████| 939/939 [03:00<00:00,  5.20it/s]

{'train_runtime': 180.4367, 'train_samples_per_second': 5.204, 'epoch': 3.0}


In [12]:
model = model.cpu()
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [18]:
test_dataset = load_dataset("imdb",split="test[5000:10000]")
test_examples= test_dataset[0:10]

Using the latest cached version of the module from C:\Users\LEOVO\.cache\huggingface\modules\datasets_modules\datasets\imdb\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a (last modified on Wed Aug 11 17:01:21 2021) since it couldn't be found locally at imdb\imdb.py or remotely (ConnectionError).
Using the latest cached version of the module from C:\Users\LEOVO\.cache\huggingface\modules\datasets_modules\datasets\imdb\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a (last modified on Wed Aug 11 17:01:21 2021) since it couldn't be found locally at imdb\imdb.py or remotely (ConnectionError).
Reusing dataset imdb (C:\Users\LEOVO\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


{'text': ["I saw Two Hands back in Sydney a few years ago and it instantly became one of my all-time favourite films. It's got action, adventure, comedy and romance all rolled up into one (and a bit of zen thrown in for good measure). Like much Australian film, the plot is easy to follow yet wonderfully engaging, and Jordan should justly feel proud of his work.<br /><br />Anyway, it was on TV just now on Channel 4 in London, and my two favourite comedy scenes of not just this movie, but indeed any movie, had been cut out! So if you watch this movie, make sure it's the original version.",
  "I'm afraid I must disagree with Mr. Radcliffe, as although he is correct in saying this isn't a comedy, it has many other merits. The plot is a little mad at parts, but I believe it it all fits together nicely, creating a satisfying, enjoyable film. The last scene was rather abysmal compared to the rest of the film, but the actual ending of the plot a few scenes previously is very interesting, showi

In [19]:
result1 = classifier(test_examples['text'])
result1

[{'label': 'LABEL_1', 'score': 0.9642220139503479},
 {'label': 'LABEL_1', 'score': 0.6508885025978088},
 {'label': 'LABEL_1', 'score': 0.7734971046447754},
 {'label': 'LABEL_1', 'score': 0.9583570957183838},
 {'label': 'LABEL_1', 'score': 0.9231916069984436},
 {'label': 'LABEL_1', 'score': 0.6773099303245544},
 {'label': 'LABEL_0', 'score': 0.9083000421524048},
 {'label': 'LABEL_1', 'score': 0.8726427555084229},
 {'label': 'LABEL_1', 'score': 0.785653829574585},
 {'label': 'LABEL_0', 'score': 0.5686978697776794}]

In [33]:
example_texts=[i[0:MAX_LENGTH*7] for i in test_examples['text']]
result1=classifier(example_texts)
example_label=test_examples['label']
# example_pred=[]
# for i in result1:
#     if i['label']=="LABEL_0":
#         example_pred.append(0)
#     else:
#         example_pred.append(1)
example_pred=[int(i['label'].split('_')[1]) for i in result1]
print(accuracy_score(example_label,example_pred))

0.8


In [34]:
test_texts=[i[0:MAX_LENGTH*2] for i in test_dataset['text']]
temp_pred=classifier(test_texts)
y_test=test_dataset['label']
y_pred=[int(i['label'].split('_')[1]) for i in temp_pred]
print(accuracy_score(y_test,y_pred))

0.8312
